In [110]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from rioxarray import merge
from osgeo import gdal
from joblib import Parallel, delayed
import sqlite3
os.environ['GDAL_NUM_THREADS'] = 'ALL_CPUS' # probably not necessary as does not trigger anything in gdal_translate or gdal2tiles

In [127]:
rgi = 9
version = '62'
MAX_VAL_MAPBOX = 800
cmap = plt.get_cmap('turbo', 256)
PATH_TIFFS_IN = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI{version}/rgi{rgi}"
tif_filenames = [os.path.basename(file) for file in glob.glob(f'{PATH_TIFFS_IN}/*.tif')]
output_directory = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI{version}/mapbox_rgi{rgi}/"
print(f"We will process {len(tif_filenames)} files from {PATH_TIFFS_IN}")
n_jobs = 8  # Number of parallel processes

We will process 1069 files from /media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI62/rgi9


In [128]:
# Process individual TIF files and save them as .tif in the temporary /tmp folder (we will remove them in the end)

def process_tif(tif_name, path_tiffs_in, cmap, max_val_mapbox):
    """Function to process a single tif file."""
    tif = rioxarray.open_rasterio(f"{path_tiffs_in}/{tif_name}").sel(band=1)  # EPSG:4326
    tif = tif.squeeze().rio.reproject(dst_crs="EPSG:3857").clip(0, max_val_mapbox).fillna(0)
    
    rgba_data = cmap(tif.values / max_val_mapbox) * 255
    #rgba_data[:, :, 3] = 127  # Set alpha to half transparency
    #rgba_data[tif.values == 0] = 0
    
    # Set alpha to fully transparent (0) where data is bad (0)
    rgba_data[tif.values == 0] = [0, 0, 0, 0]  # Fully transparent for bad data
    
    rgba_data = rgba_data.astype(np.uint8)
    
    rgb_data_array = xr.DataArray(
        rgba_data, dims=('y', 'x', 'band'),
        coords={'x': tif.coords['x'], 'y': tif.coords['y']}
    ).transpose('band', 'y', 'x')
    
    rgb_data_array.rio.write_crs("EPSG:3857", inplace=True)
    rgb_data_array.rio.write_nodata(0, inplace=True)
    
    # Skip 1-pixel glaciers
    if rgb_data_array.shape[1] == 1 or rgb_data_array.shape[2] == 1:
        return None
    
    # Save to a temporary file and return its path
    temp_file = tempfile.NamedTemporaryFile(suffix=".tif", delete=False).name
    rgb_data_array.rio.to_raster(temp_file, compress="deflate")
    return temp_file

# Parallel processing
processed_tifs = Parallel(n_jobs=n_jobs)(
    delayed(process_tif)(tif_name, PATH_TIFFS_IN, cmap, MAX_VAL_MAPBOX) 
    for tif_name in tqdm(tif_filenames, total=len(tif_filenames))
)

# Remove None values (from skipped 1-pixel glaciers)
processed_tifs = [tif for tif in processed_tifs if tif is not None]
print(f"We have no. tif to process: {len(processed_tifs)}")

 19%|███████▌                               | 208/1069 [00:01<00:06, 143.32it/s]/home/maffe/anaconda3/envs/skynetenv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████████████████████████████████| 1069/1069 [00:08<00:00, 125.78it/s]


We have no. tif to process: 1069


In [129]:
vrt_path = "temp.vrt"  # Temporary VRT file
geojson_output = f"rgi{rgi}.geojson"
mbtiles_output = f"rgi{rgi}.mbtiles"  # Output MBTiles file

try:
    # Step 1: Create a Virtual Raster (VRT)
    print(f'Begin VRT creation')
    t1_0 = time.time()
    gdal.BuildVRT(vrt_path, processed_tifs)
    if os.path.exists(vrt_path):
        print(f"{vrt_path} successfully created.")
        vrt_size = os.path.getsize(vrt_path)
        print(f"Size of {vrt_path}: {vrt_size / 1024:.2f} KB")
    else:
        raise FileNotFoundError(f"Failed to create {vrt_path}.")
    print(f"End VRT creation in {time.time()-t1_0:.2f} sec")
    
    # Step 2: Convert VRT to MBTiles using gdal_translate
    translate_command = [
        'gdal_translate',
        '-mask', '4',
        '-of', 'MBTILES',  # Output format MBTiles
        '-co', 'TILE_FORMAT=PNG',  # Tile format (PNG or JPEG)
        '-co', 'ZOOM_LEVEL_STRATEGY=LOWER',  # The nearest lower zoom level (less detailed) is chosen.
        vrt_path, mbtiles_output
    ]

    print(f'Begin MBTiles generation')
    t2_0 = time.time()
    subprocess.run(translate_command, check=True)
    subprocess.run(['gdaladdo', '-r', 'bilinear', mbtiles_output, '2', '4', '8', '16', '32', '64', '128'], check=True)
    print(f"MBTiles file created at: {mbtiles_output} in {time.time()-t2_0:.2f} sec")

    # Step 3: Update minzoom and maxzoom in Metadata
    print("Updating minzoom and maxzoom in MBTiles metadata...")
    min_zoom = 3  # Set your desired minimum zoom level
    max_zoom = 10  # Set your desired maximum zoom level
    conn = sqlite3.connect(mbtiles_output)
    cursor = conn.cursor()
    cursor.execute("UPDATE metadata SET value = ? WHERE name = 'minzoom';", (min_zoom,))
    cursor.execute("UPDATE metadata SET value = ? WHERE name = 'maxzoom';", (max_zoom,))
    conn.commit()
    conn.close()
    print(f"Updated minzoom to {min_zoom} and maxzoom to {max_zoom} in metadata.")

finally:
    # Cleanup temporary files
    for file_path in [vrt_path]:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted {file_path}.")

Begin VRT creation
temp.vrt successfully created.
Size of temp.vrt: 1935.42 KB
End VRT creation in 0.89 sec
Begin MBTiles generation
Input file size is 84301, 52594
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
MBTiles file created at: rgi9.mbtiles in 113.04 sec
Updating minzoom and maxzoom in MBTiles metadata...
Updated minzoom to 3 and maxzoom to 10 in metadata.
Deleted temp.vrt.


In [130]:
# If something is left in /tmp remove any .tif file (it should not be necessary)

# Find all .tif files in /tmp
junk_tif_files = glob.glob("/tmp/*.tif")

# Calculate total size of the .tif files
total_size_mb = sum(os.path.getsize(f) for f in junk_tif_files if os.path.isfile(f)) / (1024 ** 2)
num_files = len(junk_tif_files)

# Delete each file
for file_path in junk_tif_files:
    os.remove(file_path)
    #print(f"Deleted {file_path}")

print(f"Deleted {num_files} .tif files, freeing {total_size_mb:.2f} MB.")

Deleted 1069 .tif files, freeing 112.51 MB.
